<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/001_EDA_(Dados_do_Portal_da_cidad%C3%A2nia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Em construção - Notebook ainda não concluído



In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.8.2'